Connected to base (Python 3.13.2)

In [16]:
import os
import requests
import time
import pandas as pd

# read secrets from environment
API_KEY = os.environ.get('API_KEY')
API_URL = os.environ.get('API_URL')

if not API_KEY or not API_URL:
    raise ValueError("Missing required tokens in environment variables.")

headers = {
    "Authorization": API_KEY
}

def get_timestamps_past_7_days():
    # Current time (end timestamp) in milliseconds
    end_ts = int(time.time() * 1000)
    # Start timestamp = 7 days ago in milliseconds
    start_ts = end_ts - (7 * 24 * 60 * 60 * 1000)
    return start_ts, end_ts

def fetch_telemetry():
    start_ts, end_ts = get_timestamps_past_7_days()
    
    params = {
        "keys": "temperature,electrical_conductivity,ph",
        "startTs": str(start_ts),
        "endTs": str(end_ts),
        "interval": "10800000",  # 3-hour interval
        "agg": "AVG",
        "orderBy": "ASC"
    }
    
    response = requests.get(API_URL, headers=headers, params=params)
    if response.status_code == 200:
        print("Response received from Edenic API: data")
        data = response.json()
        return data
    else:
        print(f"Failed to fetch telemetry data: HTTP {response.status_code}")
        print(response.text)
        return None

if __name__ == "__main__":
    data = fetch_telemetry()

Response received from Edenic API: data


In [7]:
data

{'temperature': [{'ts': 1752915428289, 'value': '31.98324022346368'},
  {'ts': 1752926228289, 'value': '32.09217877094975'},
  {'ts': 1752937028289, 'value': '31.034636871508404'},
  {'ts': 1752947828289, 'value': '30.269832402234687'},
  {'ts': 1752958628289, 'value': '29.689944134078132'},
  {'ts': 1752969428289, 'value': '29.207262569832384'},
  {'ts': 1752980228289, 'value': '28.832960893854857'},
  {'ts': 1752991028289, 'value': '28.737430167597708'},
  {'ts': 1753001828289, 'value': '28.807821229050276'},
  {'ts': 1753012628289, 'value': '28.785474860335167'},
  {'ts': 1753023428289, 'value': '28.699999999999903'},
  {'ts': 1753034228289, 'value': '28.510055865921718'},
  {'ts': 1753045028289, 'value': '28.334636871508494'},
  {'ts': 1753055828289, 'value': '28.508938547486004'},
  {'ts': 1753066628289, 'value': '28.982681564245844'},
  {'ts': 1753077428289, 'value': '29.397765363128517'},
  {'ts': 1753088228289, 'value': '29.547191011235924'},
  {'ts': 1753099028289, 'value': '2

In [24]:
# Process and export each parameter
for param_name, param_data in data.items():
    # Create DataFrame
    df = pd.DataFrame(param_data)
    
    # Convert timestamp from milliseconds to datetime
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    
    # Convert value to float (except for pH which might need special handling)
    df['value'] = df['value'].astype(float)
    
    # Rename columns
    df = df.rename(columns={
        'ts': '',
        'value': param_name
    })

    # Export to CSV
    filename = f"edenic_{param_name}.csv"
    df.to_csv(filename, index=False)
    print(f"Exported {filename} with {len(df)} records")
    
    # Show sample of exported data
    print(f"\nSample of {filename}:")
    print(df.head(2), "\n" + "-"*50 + "\n")

Exported edenic_temperature.csv with 56 records

Sample of edenic_temperature.csv:
                           temperature
0 2025-07-19 08:59:14.737    31.993258
1 2025-07-19 11:59:14.737    32.083240 
--------------------------------------------------

Exported edenic_electrical_conductivity.csv with 56 records

Sample of edenic_electrical_conductivity.csv:
                           electrical_conductivity
0 2025-07-19 08:59:14.737                 4.930000
1 2025-07-19 11:59:14.737                 4.927727 
--------------------------------------------------

Exported edenic_ph.csv with 56 records

Sample of edenic_ph.csv:
                                 ph
0 2025-07-19 08:59:14.737  9.354719
1 2025-07-19 11:59:14.737  9.355642 
--------------------------------------------------



In [19]:
df

,ts,value
0,2025-07-19 08:59:14.737,9.354719
1,2025-07-19 11:59:14.737,9.355642
2,2025-07-19 14:59:14.737,9.363128
3,2025-07-19 17:59:14.737,9.366927
4,2025-07-19 20:59:14.737,9.371397
5,2025-07-19 23:59:14.737,9.387542
6,2025-07-20 02:59:14.737,9.400000
7,2025-07-20 05:59:14.737,9.409889
8,2025-07-20 08:59:14.737,9.421732
9,2025-07-20 11:59:14.737,9.420112


In [ ]:
def transform_and_export_csv(data):
    """
    Transform the JSON data into a DataFrame and export separate CSV files for
    temperature, pH, and electrical conductivity.
    Assumes data["results"] is a list of telemetry records with keys including
    'timestamp', 'temperature', 'ph', 'electrical_conductivity'.
    """

    if not data or 'results' not in data or not data['results']:
        print("No data to process")
        return

    # Create DataFrame from results
    df = pd.DataFrame(data['results'])

    # Convert timestamp from milliseconds epoch to human-readable or keep timestamp as string
    # Example: convert ms to ISO date string (optional, or keep as-is)
    # df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Rename columns to match your CSV format, including a blank first column for timestamp
    # Here we'll make the first column empty string to match your provided example
    df.rename(columns={
        'timestamp': '',
        'ph': 'pH',
        'temperature': 'Temperature',
        'electrical_conductivity': 'EC'
    }, inplace=True)

    # Reorder columns to have the unnamed time column first (empty string), then pH, Temperature, EC
    # If any columns missing, ignore to avoid errors
    cols_to_keep = [col for col in ['', 'pH', 'Temperature', 'EC'] if col in df.columns]
    df = df[cols_to_keep]

    # Save the modified full CSV (optional)
    df.to_csv('export_mod.csv', index=False)

    # Prepare dictionary for split-export filenames
    split_files = {
        'pH': 'edenic1_ph.csv',
        'Temperature': 'edenic1_temp.csv',
        'EC': 'edenic1_ec.csv'
    }

    # Export the three CSV files: each has timestamp and one measurement column
    for col, filename in split_files.items():
        if col in df.columns:
            split_df = df[['', col]]
            split_df.to_csv(filename, index=False)
            print(f"Exported {filename}")

if __name__ == "__main__":
    data = fetch_telemetry()
    if data:
        transform_and_export_csv(data)